In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import category_encoders as ce


import joblib



Imports done


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_sub = pd.read_csv("sample_submission.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)
display(train.head())


Train shape: (59611, 24)
Test shape: (14900, 23)


,founder_id,founder_age,founder_gender,years_with_startup,founder_role,monthly_revenue_generated,work_life_balance_rating,venture_satisfaction,startup_performance_rating,funding_rounds_led,...,num_dependents,startup_stage,team_size_category,years_since_founding,remote_operations,leadership_scope,innovation_support,startup_reputation,founder_visibility,retention_status
0,8410,31,Male,19,Education,5390.0,Excellent,Medium,Average,2,...,0.0,Mid,Medium,89.0,No,No,No,Excellent,Medium,Stayed
1,64756,59,Female,4,Media,5534.0,Poor,High,Low,3,...,3.0,Mid,Medium,21.0,No,No,No,Fair,Low,Stayed
2,30257,24,Female,10,Healthcare,8159.0,Good,High,Low,0,...,3.0,Mid,Medium,74.0,No,No,No,Poor,Low,Stayed
3,65791,36,Female,7,Education,3989.0,Good,High,High,1,...,2.0,Mid,Small,50.0,Yes,No,No,Good,Medium,Stayed
4,65026,56,Male,41,Education,4821.0,NaN,NaN,Average,0,...,0.0,Senior,Medium,68.0,No,No,No,Fair,Medium,Stayed


In [ ]:



TARGET = "retention_status"
IDCOL = "founder_id"

if TARGET not in train.columns:
    raise KeyError(f"Target column '{TARGET}' not found in train columns: {list(train.columns)[:30]}")

X = train.drop(columns=[TARGET], errors="ignore").drop(columns=["founder_id"], errors="ignore")
y_raw = train[TARGET].copy()

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y_raw)

print("Classes:", list(le_target.classes_))

X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("Shapes -> X_train:", X_train.shape, "X_val:", X_val.shape, "y_train:", y_train.shape, "y_val:", y_val.shape)


Target found. ID column in train? True  ID in test? True
Classes: ['Left', 'Stayed']
Shapes -> X_train: (47688, 22) X_val: (11923, 22) y_train: (47688,) y_val: (11923,)


In [ ]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print("cat_cols:", len(cat_cols), "num_cols:", len(num_cols))

pipe_te_df = Pipeline([
    ("targ_enc", ce.TargetEncoder(cols=cat_cols, smoothing=0.2)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", SVC(kernel="rbf", probability=False, random_state=42))
])
pipe_te_df 


cat_cols: 15 num_cols: 7


,steps,"[('targ_enc', ...), ('imputer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,"['founder_gender', 'founder_role', ...]"
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20


In [ ]:
SAMPLE_FRAC = 1.0   
if SAMPLE_FRAC < 1.0:
    X_train_sample = X_train.sample(frac=SAMPLE_FRAC, random_state=42)
    y_train_sample = y_train[X_train_sample.index]
    print("Using sampled training set:", X_train_sample.shape)
else:
    X_train_sample = X_train
    y_train_sample = y_train
    print("Using full training set:", X_train_sample.shape)

Using full training set: (47688, 22)


In [ ]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric cols:", len(num_cols), "Categorical cols:", len(cat_cols))

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor_keras = ColumnTransformer(
    transformers=[
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)
    ],
    remainder='drop'
)
preprocessor_keras.fit(X)

X_all = preprocessor_keras.transform(X)  
X_test_all = preprocessor_keras.transform(test.drop(columns=[IDCOL], errors='ignore'))

print("X_all shape:", X_all.shape)
print("X_test_all shape:", X_test_all.shape)

X_train_k, X_val_k, y_train_k, y_val_k = train_test_split(
    X_all, y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)

print("Train split:", X_train_k.shape, "Val split:", X_val_k.shape)



Numeric cols: 7 Categorical cols: 15
X_all shape: (59611, 56)
X_test_all shape: (14900, 56)
Train split: (47688, 56) Val split: (11923, 56)


In [ ]:
input_dim = X_train_k.shape[1]
print("Input dimension:", input_dim)

classes = np.unique(y_encoded)
class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_encoded
)
class_weight = {int(c): float(w) for c, w in zip(classes, class_weights_array)}
print("Class weights:", class_weight)

def build_mlp_model():
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid') 
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_mlp_model()

callbacks = [
    keras.callbacks.EarlyStopping(
        patience=5,
        restore_best_weights=True,
        monitor='val_loss'
    ),
    keras.callbacks.ReduceLROnPlateau(
        patience=3,
        factor=0.5,
        min_lr=1e-5,
        monitor='val_loss'
    )
]

print("\nTraining Keras MLP (with class_weight) on train/val split")
history = model.fit(
    X_train_k, y_train_k,
    validation_data=(X_val_k, y_val_k),
    epochs=50,
    batch_size=256,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=2
)

print("\nTuning threshold on validation")
val_probs = model.predict(X_val_k, verbose=0).ravel()

thresholds = np.linspace(0.35, 0.65, 31)
best_t = 0.5
best_acc = 0

for t in thresholds:
    preds = (val_probs >= t).astype(int)
    acc = accuracy_score(y_val_k, preds)
    if acc > best_acc:
        best_acc = acc
        best_t = t

print("Best threshold on val:", best_t)
print("Best val accuracy:", round(best_acc, 5))
model_full = build_mlp_model()

es_full = keras.callbacks.EarlyStopping(
    patience=4,
    restore_best_weights=True,
    monitor='loss'
)

history_full = model_full.fit(
    X_all, y_encoded,
    epochs=len(history.history['loss']),
    batch_size=256,
    class_weight=class_weight,
    callbacks=[es_full],
    verbose=2
)

print("\nPredicting on test")
test_probs = model_full.predict(X_test_all, verbose=0).ravel()
preds_num = (test_probs >= best_t).astype(int)

preds_str = le_target.inverse_transform(preds_num)

if "founder_id" in test.columns:
    ids = test["founder_id"].values
else:
    ids = np.arange(len(preds_str))

submission_keras = pd.DataFrame({
    "founder_id": ids,
    "retention_status": preds_str
})

out_path = "submission_keras_mlp.csv"
submission_keras.to_csv(out_path, index=False)


print("\nSubmission saved:", out_path)
print("Used threshold:", best_t)
print("Used class weights:", class_weight)


TensorFlow version: 2.20.0
Num GPUs available: 0
Input dimension: 56
Class weights: {0: 1.0514887462075777, 1: 0.9533184071645611}

🚀 Training Keras MLP (with class_weight) on train/val split...
Epoch 1/50
187/187 - 2s - 10ms/step - accuracy: 0.7131 - loss: 0.5463 - val_accuracy: 0.7432 - val_loss: 0.5007 - learning_rate: 0.0010
Epoch 2/50
187/187 - 0s - 2ms/step - accuracy: 0.7421 - loss: 0.5049 - val_accuracy: 0.7450 - val_loss: 0.4933 - learning_rate: 0.0010
Epoch 3/50
187/187 - 0s - 3ms/step - accuracy: 0.7463 - loss: 0.4964 - val_accuracy: 0.7460 - val_loss: 0.4907 - learning_rate: 0.0010
Epoch 4/50
187/187 - 0s - 2ms/step - accuracy: 0.7490 - loss: 0.4920 - val_accuracy: 0.7420 - val_loss: 0.4909 - learning_rate: 0.0010
Epoch 5/50
187/187 - 0s - 2ms/step - accuracy: 0.7491 - loss: 0.4893 - val_accuracy: 0.7404 - val_loss: 0.4963 - learning_rate: 0.0010
Epoch 6/50
187/187 - 0s - 2ms/step - accuracy: 0.7494 - loss: 0.4857 - val_accuracy: 0.7441 - val_loss: 0.4898 - learning_rate: 0

In [ ]:
# CELL 2 — Refit best SVM on FULL data and create submission

import numpy as np, pandas as pd, joblib

# 1) Get best SVM pipeline from search
best_svm_pipe = rnd_svm.best_estimator_
print("Using best SVM params:", rnd_svm.best_params_)

# 2) Fit on FULL training data
print("\n🚀 Fitting best SVM on FULL training data...")
best_svm_pipe.fit(X, y_encoded)
print("✅ Full-data SVM fit complete.")

# 3) Prepare test features
test_features = test.drop(columns=[IDCOL], errors="ignore")

# 4) Predict encoded labels
svm_preds_num = best_svm_pipe.predict(test_features)

# 5) Convert back to original string labels
svm_preds_str = le_target.inverse_transform(svm_preds_num)

# 6) Build submission
if IDCOL in test.columns:

    ids = test[IDCOL].values
else:
    ids = np.arange(len(svm_preds_str))

submission_svm_tuned = pd.DataFrame({
    "founder_id": ids,
    "retention_status": svm_preds_str
})

out_svm = "/content/submission_svm_tuned.csv"
submission_svm_tuned.to_csv(out_svm, index=False)

joblib.dump(best_svm_pipe, "/content/svm_tuned_full.joblib")

print("\n✅ Tuned SVM submission saved:", out_svm)
print("✅ Tuned SVM model saved:", "/content/svm_tuned_full.joblib")
print("✅ Submission rows:", len(submission_svm_tuned))


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.exceptions import NotFittedError
import joblib

X_full = train_df.drop(columns=['retention_status'])
y_full = train_df['retention_status'].map({'Retained':1,'Left':0})

try:
    n_features_after_preproc = preprocessor.transform(X_full).shape[1]
except NotFittedError:
    preprocessor.fit(X_full)
    n_features_after_preproc = preprocessor.transform(X_full).shape[1]

n_pca = min(40, n_features_after_preproc)

lr_pipe = Pipeline([
    ('preprocess', preprocessor),
    ('pca', PCA(n_components=n_pca, random_state=42)),
    ('clf', LogisticRegression(solver='saga', penalty='l2', C=1.0, max_iter=1000, class_weight='balanced', random_state=42, n_jobs=-1))
])

lr_pipe.fit(X_full, y_full)

test_ids = test_df['founder_id']
test_X = test_df.drop(columns=['founder_id']).copy()
test_preds = lr_pipe.predict(test_X)
label_map = {1:'Stayed', 0:'Left'}
submission = pd.DataFrame({'founder_id': test_ids, 'retention_status': [label_map[int(p)] for p in test_preds]})
submission.to_csv('/content/submission_lr.csv', index=False, encoding='utf-8')
joblib.dump(lr_pipe, '/content/best_logreg.joblib')
print("Saved /content/submission_lr.csv and /content/best_logreg.joblib")
print(submission['retention_status'].value_counts())
